# OLS on a System of Regressions


This notebook illustrates how to estimate a system of regressions with OLS - and to test (coefficients) across the regressions. (SURE means Seemingly Unrelated Regression Equations.)

## Load Packages and Extra Functions

In [1]:
using Printf, DelimitedFiles, Statistics, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/CovNWFn.jl")

CovNWFn

## Loading Data

In [2]:
x    = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
Rme  = x[:,2]                #market excess return
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                 #no. obs and  no. test assets

(388, 5)

## A Function for Joint Estimation of Several Regressions (OLS)


Consider the linear regression

$
y_{it}=\beta_i^{\prime}x_{t}+\varepsilon_{it}, 
$

where $i=1,2,..,n$ indicates $n$ different dependent variables. The regressors are the *same* across the regressions. 

The next cell defines a function for this estimation.

In [3]:
"""
    OlsSureFn(Y,X,NWQ=false,m=0)

LS of Y on X; where Y is Txn, and X is the same for all regressions

# Usage
(b,res,Yhat,Covb,R2) = OlsSureFn(Y,X,NWQ,m)

# Input
- `Y::Matrix`:     Txn, the n dependent variables
- `X::Matrix`:     Txk matrix of regressors (including deterministic ones)
- `NWQ:Bool`:      if true, then Newey-West's covariance matrix is used, otherwise Gauss-Markov
- `m::Int`:        scalar, bandwidth in Newey-West

# Output
- `b::Matrix`:     n*kx1, regression coefficients
- `u::Matrix`:     Txn, residuals Y - Yhat
- `Yhat::Matrix`:  Txn, fitted values X*b
- `V::Matrix`:     covariance matrix of vec(b)
- `R2::Vector`:    n-vector, R2 values

"""
function OlsSureFn(Y,X,NWQ=false,m=0)

    (T,n) = (size(Y,1),size(Y,2))
    k     = size(X,2)

    b     = X\Y
    Yhat  = X*b
    u     = Y - Yhat

    g     = zeros(T,n*k)
    for i = 1:n
      vv      = (1+(i-1)*k):(i*k)   #1:k,(1+k):2k,...
      g[:,vv] = X.*u[:,i]           #moment conditions for Y[:,i] regression
    end

    Sxx = X'X
    if NWQ
        S     = CovNWFn(g,m)            #Newey-West covariance matrix
        Sxx_1 = kron(I(n),inv(Sxx))
        V     = Sxx_1 * S * Sxx_1
    else
        V = kron(cov(u),inv(Sxx))      #traditional covariance matrix, Gauss-Markov 
    end    
    
    R2   = 1 .- var(u,dims=1)./var(Y,dims=1)

    return b, u, Yhat, V, R2

end

OlsSureFn

In [4]:
(b,u,yhat,V,R2) = OlsSureFn(Re,[ones(T) Rme])
Stdb   = sqrt.(reshape(diag(V),2,n))          #V: 1:2 are for eq 1, 3:4 for eq 2,...
tstat  = b./Stdb

printblue("CAPM regressions")
colNames = [string("asset ",i) for i=1:n]
rowNames = ["c","Rme"]

println("coeffs")
printmat(b,colNames=colNames,rowNames=rowNames)

println("t-stats")
printmat(tstat,colNames=colNames,rowNames=rowNames)

CAPM regressions
coeffs
      asset 1   asset 2   asset 3   asset 4   asset 5
c      -0.504     0.153     0.305     0.279     0.336
Rme     1.341     1.169     0.994     0.943     0.849

t-stats
      asset 1   asset 2   asset 3   asset 4   asset 5
c      -1.656     1.031     2.471     2.163     2.073
Rme    20.427    36.534    37.298    33.848    24.279



## Testing Across Regressions

To test across regressions, we first stack the point estimates into a vector by `vec(b)`. 

In [5]:
vecb = vec(b)
println("stacking the coeffs into a vector: ")
printmat(vec(b))


R = [1 0 -1 0 zeros(1,2*n-4)]           #Testing if the intercepts are
                                        #the same for asset 1 and 2
println("The R matrix")
printmat(R,width=4,prec=0)

J = size(R,1)
printlnPs("The number of hypotheses that we test: $J \n")

println("R*vec(b)")
printmat(R*vec(b))

stacking the coeffs into a vector: 
    -0.504
     1.341
     0.153
     1.169
     0.305
     0.994
     0.279
     0.943
     0.336
     0.849

The R matrix
   1   0  -1   0   0   0   0   0   0   0

The number of hypotheses that we test: 1 

R*vec(b)
    -0.657



In [6]:
Γ = R*V*R'
test_stat = (R*vec(b))'inv(Γ)*(R*vec(b))

critval = quantile(Chisq(J),0.9)          #10% critical value

printmat([test_stat critval],colNames=["test stat","10% crit val"],width=15)

      test stat   10% crit val
          6.816          2.706

